In [2]:
# !pip install prefect
# prefect config set PREFECT_API_URL=http://192.168.24.254:4200/api

# Flow and task configuration

## Attaching metadata to flows

In [1]:
import os
from prefect import flow

@flow(
    name="My Example Flow", 
    description="An example flow for a tutorial.",
    version=os.getenv("GIT_COMMIT_SHA")
)
def my_flow():
    # run tasks and subflows
    pass


### Customizing flow run names

In [3]:
import datetime
from prefect import flow

@flow(flow_run_name="{name}-on-{date:%A}")
def my_flow(name: str, date: datetime.datetime):
    pass

# creates a flow run called 'marvin-on-Thursday'
my_flow(name="marvin", date=datetime.datetime.utcnow())


14:21:59.899 | INFO    | prefect.engine - Created flow run 'benevolent-polecat' for flow 'my-flow'

14:22:00.094 | INFO    | Flow run 'benevolent-polecat' - Finished in state Completed()

In [4]:
# Additionally this setting accepts a function that returns a string for the flow run name:
import datetime
from prefect import flow

def generate_flow_run_name():
    date = datetime.datetime.utcnow()
    return f"{date:%A}-is-a-nice-day"

@flow(flow_run_name=generate_flow_run_name)
def my_flow(name: str):
    pass

# creates a flow run called 'Thursday-is-a-nice-day'
my_flow(name="marvin")


/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/flows.py:232: UserWarning: A flow named 'my-flow' and defined at '/var/folders/dv/0d5qzfx94lg3j1h65v6yghxw0000gp/T/ipykernel_55674/628841591.py:9' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


14:22:52.183 | INFO    | prefect.engine - Created flow run 'lilac-coua' for flow 'my-flow'

14:22:52.351 | INFO    | Flow run 'lilac-coua' - Finished in state Completed()

In [6]:
# If you need access to information about the flow, use the prefect.runtime module. For example

from prefect import flow
from prefect.runtime import flow_run

def generate_flow_run_name():
    flow_name = flow_run.flow_name

    parameters = flow_run.parameters
    name = parameters["name"]
    limit = parameters["limit"]

    return f"{flow_name}-with-{name}-and-{limit}"

@flow(flow_run_name=generate_flow_run_name)
def my_flow(name: str, limit: int = 100):
    pass

# creates a flow run called 'my-flow-with-marvin-and-100'
my_flow(name="marvin")


/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/flows.py:232: UserWarning: A flow named 'my-flow' and defined at '/var/folders/dv/0d5qzfx94lg3j1h65v6yghxw0000gp/T/ipykernel_55674/1177650621.py:15' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


14:23:36.963 | INFO    | prefect.engine - Created flow run 'attentive-guppy' for flow 'my-flow'

14:23:37.143 | INFO    | Flow run 'attentive-guppy' - Finished in state Completed()

## Attaching metadata to tasks

In [7]:
import datetime
from prefect import flow, task

def generate_task_name():
    date = datetime.datetime.utcnow()
    return f"{date:%A}-is-a-lovely-day"

@task(
    name="My Example Task",
    description="An example task for a tutorial.",
    task_run_name=generate_task_name
)
def my_task(name):
    pass

@flow
def my_flow():
    # creates a run with a name like "Thursday-is-a-lovely-day"
    my_task(name="marvin")


/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/flows.py:232: UserWarning: A flow named 'my-flow' and defined at '/var/folders/dv/0d5qzfx94lg3j1h65v6yghxw0000gp/T/ipykernel_55674/1329679144.py:16' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [9]:
from prefect import flow
from prefect.runtime import flow_run, task_run

def generate_task_name():
    flow_name = flow_run.flow_name
    task_name = task_run.task_name

    parameters = task_run.parameters
    name = parameters["name"]
    limit = parameters["limit"]

    return f"{flow_name}-{task_name}-with-{name}-and-{limit}"

@task(
    name="my-example-task",
    description="An example task for a tutorial.",
    task_run_name=generate_task_name
)
def my_task(name: str, limit: int = 100):
    pass

@flow
def my_flow(name: str):
    # creates a run with a name like "my-flow-my-example-task-with-marvin-and-100"
    my_task(name="marvin")


/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/tasks.py:287: UserWarning: A task named 'my-example-task' and defined at '/var/folders/dv/0d5qzfx94lg3j1h65v6yghxw0000gp/T/ipykernel_55674/15213493.py:14' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


## Flow and task retries

In [11]:
from prefect import flow, task

# this tasks runs 3 times before the flow fails
@task(retries=2, retry_delay_seconds=5)
def failure():
    print('running')
    raise ValueError("bad code")

@flow
def test_retries():
    return failure()

test_retries()

/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/tasks.py:287: UserWarning: A task named 'failure' and defined at '/var/folders/dv/0d5qzfx94lg3j1h65v6yghxw0000gp/T/ipykernel_55674/3437083298.py:4' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/flows.py:232: UserWarning: A flow named 'test-retries' and defined at '/var/folders/dv/0d5qzfx94lg3j1h65v6yghxw0000gp/T/ipykernel_55674/3437083298.py:9' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


14:36:25.351 | INFO    | prefect.engine - Created flow run 'sandy-otter' for flow 'test-retries'

14:36:27.407 | INFO    | Flow run 'sandy-otter' - Created task run 'failure-0' for task 'failure'

14:36:27.410 | INFO    | Flow run 'sandy-otter' - Executing 'failure-0' immediately...

running


...

14:36:43.278 | ERROR   | Task run 'failure-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/engine.py", line 1581, in orchestrate_task_run
    result = await call.aresult()
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/_internal/concurrency/calls.py", line 181, in aresult
    return await asyncio.wrap_future(self.future)
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/_internal/concurrency/calls.py", line 194, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
  File "/var/folders/dv/0d5qzfx94lg3j1h65v6yghxw0000gp/T/ipykernel_55674/3437083298.py", line 7, in failure
    raise ValueError("bad code")
ValueError: bad code

14:36:43.330 | ERROR   | Task run 'failure-0' - Finished in state Failed('Task run encountered an exception: ValueError: bad code\n')

14:36:43.333 | ERROR   | Flow run 'sandy-otter' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/engine.py", line 700, in orchestrate_flow_run
    result = await flow_call.aresult()
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/_internal/concurrency/calls.py", line 181, in aresult
    return await asyncio.wrap_future(self.future)
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/_internal/concurrency/calls.py", line 194, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
  File "/var/folders/dv/0d5qzfx94lg3j1h65v6yghxw0000gp/T/ipykernel_55674/3437083298.py", line 11, in test_retries
    return failure()
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/tasks.py", line 494, in __call__
    return enter_task_run_engine(
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/engine.py", line 1003, in enter_task_run_engine
    return from_sync.wait_for_call_in_loop_thread(begin_run)
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/_internal/concurrency/api.py", line 215, in wait_for_call_in_loop_thread
    return call.result()
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/_internal/concurrency/calls.py", line 173, in result
    return self.future.result(timeout=timeout)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/concurrent/futures/_base.py", line 433, in result
    return self.__get_result()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/concurrent/futures/_base.py", line 389, in __get_result
    raise self._exception
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/_internal/concurrency/calls.py", line 218, in _run_async
    result = await coro
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/engine.py", line 1168, in get_task_call_return_value
    return await future._result()
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/futures.py", line 241, in _result
    return await final_state.result(raise_on_failure=raise_on_failure, fetch=True)
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/states.py", line 91, in _get_state_result
    raise await get_state_exception(state)
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/engine.py", line 1581, in orchestrate_task_run
    result = await call.aresult()
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/_internal/concurrency/calls.py", line 181, in aresult
    return await asyncio.wrap_future(self.future)
  File "/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/_internal/concurrency/calls.py", line 194, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
  File "/var/folders/dv/0d5qzfx94lg3j1h65v6yghxw0000gp/T/ipykernel_55674/3437083298.py", line 7, in failure
    raise ValueError("bad code")
ValueError: bad code

14:36:43.364 | ERROR   | Flow run 'sandy-otter' - Finished in state Failed('Flow run encountered an exception. ValueError: bad code\n')

ValueError: bad code

## Task caching

In [13]:
from prefect import flow, task
from prefect.tasks import task_input_hash
from datetime import timedelta

@task(cache_key_fn=task_input_hash, cache_expiration=timedelta(minutes=1))
def hello_task(name_input):
    # Doing some work
    print(f"Saying hello {name_input}")
    return "hello " + name_input

@flow
def hello_flow(name_input):
    hello_task(name_input)

/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/tasks.py:287: UserWarning: A task named 'hello_task' and defined at '/var/folders/dv/0d5qzfx94lg3j1h65v6yghxw0000gp/T/ipykernel_55674/2959131724.py:5' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/flows.py:232: UserWarning: A flow named 'hello-flow' and defined at '/var/folders/dv/0d5qzfx94lg3j1h65v6yghxw0000gp/T/ipykernel_55674/2959131724.py:11' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


### Using a custom cache key function

In [14]:
from prefect import flow, task
from datetime import timedelta
import time

def cache_key_from_sum(context, parameters):
    print(parameters)
    return sum(parameters["nums"])

@task(cache_key_fn=cache_key_from_sum, cache_expiration=timedelta(minutes=1))
def cached_task(nums):
    print('running an expensive operation')
    time.sleep(3)
    return sum(nums)

@flow
def test_caching(nums):
    cached_task(nums)


## Configuring task runners

In [15]:
from prefect import flow, task
from prefect.task_runners import SequentialTaskRunner

@task
def first_task(num):
    return num + num

@task
def second_task(num):
    return num * num

@flow(name="My Example Flow",
    task_runner=SequentialTaskRunner(),
)
def my_flow(num):
    plusnum = first_task.submit(num)
    sqnum = second_task.submit(plusnum)
    print(f"add: {plusnum.result()}, square: {sqnum.result()}")

my_flow(5)


/Users/macpro2014/Documents/projects/dl_labs/.venv/lib/python3.9/site-packages/prefect/flows.py:232: UserWarning: A flow named 'My Example Flow' and defined at '/var/folders/dv/0d5qzfx94lg3j1h65v6yghxw0000gp/T/ipykernel_55674/2450218093.py:12' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


15:12:29.508 | INFO    | prefect.engine - Created flow run 'impossible-teal' for flow 'My Example Flow'

15:12:29.633 | INFO    | Flow run 'impossible-teal' - Created task run 'first_task-0' for task 'first_task'

15:12:29.636 | INFO    | Flow run 'impossible-teal' - Executing 'first_task-0' immediately...

15:12:29.741 | INFO    | Task run 'first_task-0' - Finished in state Completed()

...

15:12:29.772 | INFO    | Flow run 'impossible-teal' - Executing 'second_task-0' immediately...

15:12:29.865 | INFO    | Task run 'second_task-0' - Finished in state Completed()

add: 10, square: 100


15:12:29.897 | INFO    | Flow run 'impossible-teal' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `int`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `int`'))]